# Toy example for Radix-based for multiple conditions

In [16]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [17]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

### Try optimizing using radix for one condition first

In [25]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
mdl_ref = ijomc
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound = 0.001
not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [26]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [27]:
N_CONDS = 2

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [30]:
stacker = StackOptimizer()
stacker.stack_models(ijomc, df_conds)

In [31]:
stacker.model.optimize(solver='gurobi')

<Solution 1.75 at 0x7f8593dc05d0>

In [32]:
stacker.model_dict['fructose'].optimize()

<Solution 0.87 at 0x7f8593e71390>

In [40]:
stacker.model_dict['glucose'].optimize()
print stacker.model_dict['glucose'].reactions.query('BIOMASS')[0].x
print stacker.model_dict['glucose'].reactions.query('EX_glc')[0].x
print stacker.model_dict['glucose'].reactions.query('EX_fru')[0].x

0.873921506968
-10.0
0.0


In [42]:
stacker.model_dict['fructose'].optimize()
print stacker.model_dict['fructose'].reactions.query('BIOMASS')[0].x
print stacker.model_dict['fructose'].reactions.query('EX_glc')[0].x
print stacker.model_dict['fructose'].reactions.query('EX_fru')[0].x

0.873921506966
0.0
-10.0


In [43]:
for mdl_ind,mdl in iteritems(stacker.model_dict):
    opt = Optimizer(mdl)
    gapi = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)

In [44]:
stacker.model.optimize(solver='gurobi')

<Solution 3.50 at 0x7f8593dc07d0>

In [45]:
import numpy as np

radix = 2.
print('Radix:',radix)
powers = np.arange(-3,4)
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Discretize crowding coefficients into radix
#crowding_p = gap.metabolites.crowding
#crowding_cons = stacker.model.metabolites.query('crowding')

var_cons_dict = {}
# Get the group ID from reference model
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
for rxn_ref in crowding_ref.reactions:    
    for mdl_ind, mdl in iteritems(stacker.model_dict):
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        # Get the coefficient in the dual
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

('Radix:', 2.0)
('Powers:', array([-3, -2, -1,  0,  1,  2,  3]))
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [46]:
# %lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power, prevent_zero=True)

In [47]:
%time opt.to_radix(stacker.model, var_cons_dict, radix, powers, digits=digits, prevent_zero=False)

CPU times: user 121 ms, sys: 18.7 ms, total: 139 ms
Wall time: 124 ms


In [48]:
for rxn in stacker.model.reactions:
    rxn.objective_coefficient = 0.
for rxn in stacker.model.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [52]:
F_TOL = 0.01

mu_id = 'BIOMASS_Ecoli_core_w_GAM'

print('#'*40)
for mdl_ind,mdl in iteritems(stacker.model_dict):
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    rxn_mu = mdl.reactions.get_by_id(mu_id+'_%s'%mdl_ind)    
    rxn_mu.lower_bound = mu_meas*(1-F_TOL)
    rxn_mu.upper_bound = mu_meas*(1+F_TOL)
    mu_crowd0 = np.nan
    print('Condition=%s. Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mdl_ind, mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))
print('#'*40)

########################################
Condition=fructose. Initial mu_crowd=nan. Fitting within 1% of measured: 0.5445 <= mu <= 0.5555
Condition=glucose. Initial mu_crowd=nan. Fitting within 1% of measured: 0.7326 <= mu <= 0.7474
########################################


In [ ]:
from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(stacker.model)
milp.ModelSense = GRB.MINIMIZE
#milp.ModelSense = GRB.MAXIMIZE

milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.Params.FeasibilityTol = 1e-9
milp.Params.OptimalityTol = 1e-9
milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter FeasibilityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Changed value of parameter OptimalityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Optimize a model with 5714 rows, 2396 columns and 15912 nonzeros
Variable types: 2060 continuous, 336 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 1e+03]
  RHS range        [1e-03, 1e+03]
Presolve removed 1762 rows and 398 columns
Presolve time: 0.04s
Presolved: 3952 rows, 1998 columns, 11716 nonzeros
Variable types: 1676 continuous, 322 integer (322 binary)

Root relaxation: objective 1.589302e+00, 1535 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.58930    0   

In [51]:
sol = gurobi_solver.format_solution(milp, stacker.model)
yopt = [sol.x_dict[rxn.id] for rxn in stacker.model.reactions.query('binary_')]
print('Number of non-zero binaries: %g' % sum(yopt))
for mdl_ind,mdl in iteritems(stacker.model_dict):
    rxn_mu = stacker.model.reactions.get_by_id(mu_id+'_%s'%mdl_ind)    
    muopt = sol.x_dict[rxn_mu.id]
    print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))

Number of non-zero binaries: 0
Initial mu_crowd=nan. Fitted within 100%: 0 <= 0.873922 <= 1.1
Initial mu_crowd=nan. Fitted within 100%: 0 <= 0.873922 <= 1.48


In [ ]:
stacker.model.reactions.binary_ACALD00.metabolites

In [ ]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
#                 print yid, y
#             if abs(y)<1e-10:
#                 print yid, y
                pass
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

In [ ]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

### Plug back in to be sure

#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding_bound = ijomc.metabolites.crowding._bound
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

ijofit.optimize()
print('Mu=%g' % ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x)